<a href="https://colab.research.google.com/github/ErdemAslans/CityCoordinatesAlgorithm-for-Autonomous-Systems-Hackathon-Challenge/blob/main/CityCoordinatesAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install torch plotly


In [ ]:
# Import necessary libraries
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  # Optional
import seaborn as sns  # Optional
from scipy.spatial.distance import pdist, squareform
import random
from typing import List, Tuple, Dict
import time
from itertools import product
from IPython.display import clear_output
import plotly.graph_objects as go
import plotly.express as px

# Mount Google Drive to access the data file
from google.colab import drive
drive.mount('/content/drive')

# GPU Check
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")



Mounted at /content/drive
Using device: cuda


In [ ]:
# Define the file path
file_path = '/content/drive/My Drive/CityCoordinatesAlgortihm/coordinates.csv'

# Load the data
df = pd.read_csv(file_path, header=None)

# Split the single column into separate columns
split_data = df[0].str.split(expand=True)
split_data.columns = ['city_id', 'x', 'y']

# Convert data types
split_data['city_id'] = split_data['city_id'].astype(int)
split_data['x'] = split_data['x'].astype(float)
split_data['y'] = split_data['y'].astype(float)

print(f"Total number of cities: {len(split_data)}")
print("\nSample of the dataset:")
print(split_data.head())


Total number of cities: 532

Sample of the dataset:
   city_id       x       y
0        1  267.10  159.40
1        2   23.20   88.65
2        3  228.45   44.30
3        4  335.65  207.70
4        5  370.40  176.70


In [ ]:
# Convert coordinates to tensor and move to GPU
coords_tensor = torch.tensor(
    split_data[['x', 'y']].values,
    dtype=torch.float32
).to(device)

n_cities = len(coords_tensor)

# Calculate the distance matrix
diff = coords_tensor.unsqueeze(1) - coords_tensor.unsqueeze(0)  # Shape: (n_cities, n_cities, 2)
distances = torch.sqrt(torch.sum(diff ** 2, dim=2))  # Shape: (n_cities, n_cities)

print("Distance matrix calculated.")


Distance matrix calculated.


In [ ]:
@torch.jit.script
def nearest_neighbor_jit_optimized(distances: torch.Tensor, start_city: int) -> Tuple[torch.Tensor, float]:
    n_cities = distances.size(0)
    tour = torch.empty(n_cities + 1, dtype=torch.long, device=distances.device)
    tour[0] = start_city
    unvisited = torch.ones(n_cities, dtype=torch.bool, device=distances.device)
    unvisited[start_city] = False
    current_city = start_city
    total_length = 0.0

    for i in range(1, n_cities):
        masked_distances = distances[current_city].masked_fill(~unvisited, float('inf'))
        next_city = torch.argmin(masked_distances)
        tour[i] = next_city
        total_length += distances[current_city, next_city]
        unvisited[next_city] = False
        current_city = next_city

    tour[-1] = start_city
    total_length += distances[current_city, start_city]

    return tour, total_length

def nearest_neighbor(start_city: int = 0) -> Tuple[List[int], float]:
    """Nearest Neighbor algorithm using JIT-optimized function."""
    tour_tensor, total_length = nearest_neighbor_jit_optimized(distances, start_city)
    tour_list = tour_tensor.cpu().tolist()
    return tour_list, total_length


In [ ]:
### Multiple Starts for Nearest Neighbor
# Run the Nearest Neighbor algorithm from multiple starting cities to find a better solution.

def nearest_neighbor_multiple_starts(num_starts: int = 10) -> Tuple[List[int], float]:
    """Run Nearest Neighbor from multiple starting points and return the best tour."""
    best_tour = None
    best_length = float('inf')

    for start in range(num_starts):
        tour, length = nearest_neighbor(start_city=start)
        if length < best_length:
            best_length = length
            best_tour = tour
        print(f"Start City {start}: Tour Length = {length:.2f}")

    return best_tour, best_length


In [ ]:
### 2-Opt Optimization
#Improve the initial tour using the 2-Opt local search algorithm.

def two_opt_swap(tour: List[int], i: int, j: int) -> List[int]:
    """Perform a 2-opt swap by reversing the tour segment between indices i and j."""
    new_tour = tour[:i] + tour[i:j+1][::-1] + tour[j+1:]
    return new_tour

def calculate_tour_length(tour: List[int]) -> float:
    """Calculate the total length of the tour."""
    tour_extended = tour + [tour[0]]
    tour_tensor = torch.tensor(tour_extended, device=device, dtype=torch.long)
    indices = torch.stack([tour_tensor[:-1], tour_tensor[1:]], dim=1)
    lengths = distances[indices[:,0], indices[:,1]]
    total_length = torch.sum(lengths).item()
    return total_length

def two_opt_improvement(initial_tour: List[int], max_iterations: int = 200) -> Tuple[List[int], float]:
    """Improve the tour using the 2-Opt algorithm."""
    best_tour = initial_tour
    best_length = calculate_tour_length(best_tour)

    for iteration in range(1, max_iterations + 1):
        improvement_found = False
        # Select 100 random city indices (excluding the start/end city)
        cities = torch.randperm(len(best_tour) - 2, device=device)[:100] + 1

        for i in cities:
            i = i.item()
            # Check the next 20 cities for potential swaps
            for j in range(i + 1, min(i + 20, len(best_tour) -1)):
                new_tour = two_opt_swap(best_tour, i, j)
                new_length = calculate_tour_length(new_tour)

                if new_length < best_length:
                    best_tour = new_tour
                    best_length = new_length
                    improvement_found = True
                    break
            if improvement_found:
                break

        if iteration % 10 == 0 or iteration == 1:
            print(f"Iteration {iteration}/{max_iterations}, Current Length: {best_length:.2f}")

        if not improvement_found:
            print("No further improvement found.")
            break

    return best_tour, best_length


In [ ]:
### Ant Colony Optimization (ACO)
# Implement the Ant Colony Optimization algorithm to find an optimized tour.

def ant_colony_optimization(num_ants: int = 50, num_iterations: int = 2,
                            alpha: float = 1.0, beta: float = 5.0,
                            evaporation_rate: float = 0.5, pheromone_initial: float = 1.0) -> Tuple[List[int], float]:
    """Ant Colony Optimization algorithm for TSP."""
    pheromone = torch.full((n_cities, n_cities), pheromone_initial, device=device)
    heuristic = 1.0 / (distances + torch.eye(n_cities, device=device))  # Avoid division by zero

    best_tour = None
    best_length = float('inf')

    for iteration in range(1, num_iterations + 1):
        all_tours = []
        all_lengths = []

        for ant in range(num_ants):
            current_city = random.randint(0, n_cities - 1)
            tour = [current_city]
            visited = set(tour)

            while len(tour) < n_cities:
                current_pheromone = pheromone[current_city]
                current_heuristic = heuristic[current_city]
                probabilities = (current_pheromone ** alpha) * (current_heuristic ** beta)
                probabilities[list(visited)] = 0  # Mask visited cities

                total = probabilities.sum()
                if total == 0:
                    break  # All cities visited

                probabilities /= total
                next_city = torch.multinomial(probabilities, 1).item()
                tour.append(next_city)
                visited.add(next_city)
                current_city = next_city

            tour_length = calculate_tour_length(tour)
            all_tours.append(tour)
            all_lengths.append(tour_length)

            if tour_length < best_length:
                best_length = tour_length
                best_tour = tour

        # Evaporate pheromones
        pheromone *= (1 - evaporation_rate)

        # Update pheromones based on tours
        for tour, length in zip(all_tours, all_lengths):
            pheromone_increment = 1.0 / length
            for i in range(len(tour)):
                pheromone[tour[i]][tour[(i + 1) % n_cities]] += pheromone_increment
                pheromone[tour[(i + 1) % n_cities]][tour[i]] += pheromone_increment

        if iteration % 10 == 0 or iteration == 1:
            print(f"ACO Iteration {iteration}/{num_iterations}, Best Length: {best_length:.2f}")

    return best_tour, best_length


In [ ]:
def plot_cities_interactive_plotly():
    """Plot the cities on an interactive Plotly scatter plot."""
    fig = go.Figure(data=go.Scatter(
        x=split_data['x'],
        y=split_data['y'],
        mode='markers',
        marker=dict(color='blue', size=5, opacity=0.6),
        text=split_data['city_id'],
        hoverinfo='text'
    ))

    fig.update_layout(
        title='City Locations',
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        template='plotly_white',
        width=1000,
        height=800
    )

    fig.show()


In [ ]:
def plot_tour_plotly(tour: List[int], title: str = "TSP Tour"):
    """Plot the tour on an interactive Plotly graph."""
    tour_extended = tour + [tour[0]]
    x = split_data.loc[tour_extended, 'x'].values
    y = split_data.loc[tour_extended, 'y'].values

    fig = go.Figure()

    # Plot the tour lines
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='lines',
        line=dict(color='orange', width=1),
        name='Tour Path'
    ))

    # Plot the cities
    fig.add_trace(go.Scatter(
        x=split_data['x'],
        y=split_data['y'],
        mode='markers',
        marker=dict(color='blue', size=5, opacity=0.6),
        text=split_data['city_id'],
        hoverinfo='text',
        name='Cities'
    ))

    # Highlight the start city
    fig.add_trace(go.Scatter(
        x=[x[0]],
        y=[y[0]],
        mode='markers',
        marker=dict(color='green', size=10, symbol='star'),
        name='Start City'
    ))

    # Highlight the end city
    fig.add_trace(go.Scatter(
        x=[x[-1]],
        y=[y[-1]],
        mode='markers',
        marker=dict(color='red', size=10, symbol='star'),
        name='End City'
    ))

    fig.update_layout(
        title=f"{title}\nTotal Distance: {calculate_tour_length(tour):.2f}",
        xaxis_title='X Coordinate',
        yaxis_title='Y Coordinate',
        template='plotly_white',
        width=1000,
        height=800
    )

    fig.show()


In [ ]:
def plot_comparison_plotly(results: Dict):
    """Compare algorithm performances using Plotly."""
    algorithms = list(results.keys())
    lengths = [result['length'] for result in results.values()]
    times = [result['time'] for result in results.values()]

    # Compare Tour Lengths
    fig1 = px.bar(x=algorithms, y=lengths, labels={'x': 'Algorithm', 'y': 'Tour Length'},
                  title='Comparison of Tour Lengths', text=[f"{l:.1f}" for l in lengths],
                  color=lengths, color_continuous_scale='Viridis')
    fig1.update_traces(textposition='outside', marker_line_width=0)
    fig1.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', width=1000, height=600)
    fig1.show()

    # Compare Execution Times
    fig2 = px.bar(x=algorithms, y=times, labels={'x': 'Algorithm', 'y': 'Time (s)'},
                  title='Comparison of Execution Times', text=[f"{t:.2f}s" for t in times],
                  color=times, color_continuous_scale='Plasma')
    fig2.update_traces(textposition='outside', marker_line_width=0)
    fig2.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', width=1000, height=600)
    fig2.show()

    # Detailed Comparison Table
    print("\nDetailed Comparison:")
    print("-" * 60)
    print(f"{'Algorithm':<30} {'Tour Length':<20} {'Time (s)':<15}")
    print("-" * 60)
    for algo, result in results.items():
        print(f"{algo:<30} {result['length']:<20.2f} {result['time']:<15.2f}")


In [ ]:
# Plot city locations
print("Visualizing city distribution...")
plot_cities_interactive_plotly()


Visualizing city distribution...


In [ ]:
# Initialize results dictionary
results = {}

# Nearest Neighbor - Multiple Starts
print("\nRunning Nearest Neighbor (Multiple Starts)...")
start_time = time.time()
nn_tour, nn_length = nearest_neighbor_multiple_starts(num_starts=10)
nn_time = time.time() - start_time
results['Nearest Neighbor (Multi-Start)'] = {
    'tour': nn_tour,
    'length': nn_length,
    'time': nn_time
}
print(f"Nearest Neighbor (Multi-Start) completed in {nn_time:.2f} seconds with tour length {nn_length:.2f}")
print(f"Tour Sequence: {' -> '.join(map(str, nn_tour))}")
plot_tour_plotly(nn_tour, "TSP Tour - Nearest Neighbor (Multi-Start)")



Running Nearest Neighbor (Multiple Starts)...
Start City 0: Tour Length = 5220.83
Start City 1: Tour Length = 5502.73
Start City 2: Tour Length = 5491.21
Start City 3: Tour Length = 5484.71
Start City 4: Tour Length = 5484.86
Start City 5: Tour Length = 5589.25
Start City 6: Tour Length = 5559.45
Start City 7: Tour Length = 5479.12
Start City 8: Tour Length = 5349.73
Start City 9: Tour Length = 5569.93
Nearest Neighbor (Multi-Start) completed in 0.71 seconds with tour length 5220.83
Tour Sequence: 0 -> 207 -> 447 -> 8 -> 381 -> 317 -> 263 -> 24 -> 164 -> 316 -> 11 -> 176 -> 165 -> 177 -> 283 -> 478 -> 137 -> 363 -> 84 -> 475 -> 229 -> 38 -> 45 -> 264 -> 457 -> 333 -> 369 -> 124 -> 89 -> 345 -> 232 -> 21 -> 152 -> 72 -> 531 -> 225 -> 96 -> 166 -> 441 -> 426 -> 509 -> 123 -> 374 -> 219 -> 352 -> 450 -> 86 -> 347 -> 384 -> 58 -> 487 -> 54 -> 430 -> 51 -> 285 -> 511 -> 92 -> 175 -> 85 -> 284 -> 209 -> 386 -> 64 -> 108 -> 106 -> 135 -> 322 -> 409 -> 158 -> 528 -> 346 -> 110 -> 26 -> 440 ->

In [ ]:
# Nearest Neighbor + 2-Opt
print("\nRunning Nearest Neighbor + 2-Opt Optimization...")
start_time = time.time()
nn_2opt_tour, nn_2opt_length = two_opt_improvement(nn_tour, max_iterations=200)
nn_2opt_time = time.time() - start_time
results['NN + 2-opt'] = {
    'tour': nn_2opt_tour,
    'length': nn_2opt_length,
    'time': nn_time + nn_2opt_time
}
print(f"Nearest Neighbor + 2-Opt completed in {nn_2opt_time:.2f} seconds with tour length {nn_2opt_length:.2f}")
print(f"Tour Sequence: {' -> '.join(map(str, nn_2opt_tour))}")
plot_tour_plotly(nn_2opt_tour, "TSP Tour - NN + 2-opt")




Running Nearest Neighbor + 2-Opt Optimization...
Iteration 1/200, Current Length: 5217.21
Iteration 10/200, Current Length: 5182.78
Iteration 20/200, Current Length: 5169.86
Iteration 30/200, Current Length: 5130.62
Iteration 40/200, Current Length: 5108.41
Iteration 50/200, Current Length: 5090.26
No further improvement found.
Nearest Neighbor + 2-Opt completed in 3.54 seconds with tour length 5057.50
Tour Sequence: 0 -> 207 -> 447 -> 8 -> 381 -> 317 -> 263 -> 24 -> 164 -> 316 -> 11 -> 176 -> 165 -> 177 -> 283 -> 478 -> 137 -> 363 -> 84 -> 475 -> 229 -> 38 -> 45 -> 264 -> 457 -> 333 -> 369 -> 124 -> 89 -> 345 -> 232 -> 21 -> 152 -> 72 -> 531 -> 225 -> 96 -> 166 -> 441 -> 426 -> 509 -> 123 -> 374 -> 219 -> 352 -> 450 -> 86 -> 347 -> 384 -> 58 -> 487 -> 54 -> 430 -> 51 -> 285 -> 511 -> 92 -> 175 -> 85 -> 284 -> 209 -> 386 -> 64 -> 106 -> 108 -> 135 -> 322 -> 409 -> 158 -> 528 -> 346 -> 110 -> 26 -> 440 -> 523 -> 256 -> 455 -> 169 -> 250 -> 452 -> 191 -> 33 -> 373 -> 167 -> 258 -> 376 -

In [ ]:
# Ant Colony Optimization
print("\nRunning Ant Colony Optimization (ACO)...")
start_time = time.time()
aco_tour, aco_length = ant_colony_optimization()
aco_time = time.time() - start_time
results['Ant Colony Optimization'] = {
    'tour': aco_tour,
    'length': aco_length,
    'time': aco_time
}
print(f"Ant Colony Optimization completed in {aco_time:.2f} seconds with tour length {aco_length:.2f}")
print(f"Tour Sequence: {' -> '.join(map(str, aco_tour))}")
plot_tour_plotly(aco_tour, "TSP Tour - Ant Colony Optimization")



Running Ant Colony Optimization (ACO)...
ACO Iteration 1/2, Best Length: 6445.23
Ant Colony Optimization completed in 22.49 seconds with tour length 6380.30
Tour Sequence: 4 -> 313 -> 197 -> 337 -> 193 -> 139 -> 154 -> 407 -> 246 -> 506 -> 503 -> 269 -> 488 -> 43 -> 339 -> 79 -> 342 -> 6 -> 443 -> 233 -> 77 -> 406 -> 260 -> 29 -> 438 -> 490 -> 351 -> 147 -> 280 -> 301 -> 121 -> 119 -> 226 -> 227 -> 518 -> 434 -> 405 -> 255 -> 358 -> 141 -> 249 -> 19 -> 262 -> 429 -> 527 -> 368 -> 462 -> 520 -> 75 -> 179 -> 118 -> 500 -> 73 -> 127 -> 432 -> 125 -> 81 -> 95 -> 216 -> 319 -> 356 -> 388 -> 371 -> 172 -> 334 -> 392 -> 338 -> 415 -> 330 -> 294 -> 35 -> 460 -> 131 -> 181 -> 199 -> 196 -> 251 -> 126 -> 282 -> 348 -> 248 -> 276 -> 265 -> 360 -> 364 -> 321 -> 465 -> 180 -> 516 -> 315 -> 307 -> 350 -> 379 -> 188 -> 171 -> 236 -> 222 -> 112 -> 431 -> 55 -> 306 -> 65 -> 212 -> 182 -> 435 -> 80 -> 507 -> 328 -> 71 -> 162 -> 132 -> 451 -> 290 -> 257 -> 370 -> 143 -> 253 -> 529 -> 508 -> 163 -> 76 ->

In [ ]:
# Compare all algorithm results
print("\nComparing All Algorithms:")
plot_comparison_plotly(results)



Comparing All Algorithms:



Detailed Comparison:
------------------------------------------------------------
Algorithm                      Tour Length          Time (s)       
------------------------------------------------------------
Nearest Neighbor (Multi-Start) 5220.83              1.32           
NN + 2-opt                     5086.96              5.34           
Ant Colony Optimization        6338.44              22.46          


In [ ]:
# Print Detailed Results
print("\nFinal Results:")
print("-" * 60)
for algorithm, result in results.items():
    print(f"\n{algorithm}:")
    print(f"Tour Length: {result['length']:.2f}")
    print(f"Computation Time: {result['time']:.2f} seconds")
    print(f"Tour Sequence: {' -> '.join(map(str, result['tour']))}")



Final Results:
------------------------------------------------------------

Nearest Neighbor (Multi-Start):
Tour Length: 5220.83
Computation Time: 0.71 seconds
Tour Sequence: 0 -> 207 -> 447 -> 8 -> 381 -> 317 -> 263 -> 24 -> 164 -> 316 -> 11 -> 176 -> 165 -> 177 -> 283 -> 478 -> 137 -> 363 -> 84 -> 475 -> 229 -> 38 -> 45 -> 264 -> 457 -> 333 -> 369 -> 124 -> 89 -> 345 -> 232 -> 21 -> 152 -> 72 -> 531 -> 225 -> 96 -> 166 -> 441 -> 426 -> 509 -> 123 -> 374 -> 219 -> 352 -> 450 -> 86 -> 347 -> 384 -> 58 -> 487 -> 54 -> 430 -> 51 -> 285 -> 511 -> 92 -> 175 -> 85 -> 284 -> 209 -> 386 -> 64 -> 108 -> 106 -> 135 -> 322 -> 409 -> 158 -> 528 -> 346 -> 110 -> 26 -> 440 -> 3 -> 49 -> 148 -> 150 -> 502 -> 130 -> 258 -> 167 -> 373 -> 33 -> 191 -> 452 -> 250 -> 169 -> 455 -> 256 -> 523 -> 483 -> 403 -> 34 -> 93 -> 289 -> 504 -> 39 -> 10 -> 55 -> 306 -> 431 -> 221 -> 376 -> 503 -> 506 -> 407 -> 246 -> 338 -> 392 -> 334 -> 356 -> 388 -> 319 -> 216 -> 95 -> 81 -> 125 -> 432 -> 127 -> 415 -> 294 -> 4

In [ ]:
# Find the algorithm with the shortest tour
best_algorithm = min(results.items(), key=lambda x: x[1]['length'])
best_tour = best_algorithm[1]['tour']
best_length = best_algorithm[1]['length']

# Save the results to a text file
with open('/content/drive/My Drive/CityCoordinatesAlgortihm/shortest_tour.txt', 'w') as f:
    f.write(f"Best Algorithm: {best_algorithm[0]}\n")
    f.write(f"Tour Length: {best_length:.2f}\n")
    f.write(f"Tour Sequence: {' -> '.join(map(str, best_tour))}\n")

print(f"\nShortest tour and its length have been saved to 'shortest_tour.txt'.")

# Optionally, display the content of the text file
with open('/content/drive/My Drive/CityCoordinatesAlgortihm/shortest_tour.txt', 'r') as f:
    tour_info = f.read()

print("\nShortest Tour Details:")
print(tour_info)



Shortest tour and its length have been saved to 'shortest_tour.txt'.

Shortest Tour Details:
Best Algorithm: NN + 2-opt
Tour Length: 5057.50
Tour Sequence: 0 -> 207 -> 447 -> 8 -> 381 -> 317 -> 263 -> 24 -> 164 -> 316 -> 11 -> 176 -> 165 -> 177 -> 283 -> 478 -> 137 -> 363 -> 84 -> 475 -> 229 -> 38 -> 45 -> 264 -> 457 -> 333 -> 369 -> 124 -> 89 -> 345 -> 232 -> 21 -> 152 -> 72 -> 531 -> 225 -> 96 -> 166 -> 441 -> 426 -> 509 -> 123 -> 374 -> 219 -> 352 -> 450 -> 86 -> 347 -> 384 -> 58 -> 487 -> 54 -> 430 -> 51 -> 285 -> 511 -> 92 -> 175 -> 85 -> 284 -> 209 -> 386 -> 64 -> 106 -> 108 -> 135 -> 322 -> 409 -> 158 -> 528 -> 346 -> 110 -> 26 -> 440 -> 523 -> 256 -> 455 -> 169 -> 250 -> 452 -> 191 -> 33 -> 373 -> 167 -> 258 -> 376 -> 221 -> 431 -> 306 -> 55 -> 10 -> 504 -> 39 -> 289 -> 93 -> 34 -> 403 -> 483 -> 3 -> 49 -> 148 -> 150 -> 502 -> 130 -> 503 -> 506 -> 407 -> 246 -> 338 -> 392 -> 334 -> 388 -> 356 -> 319 -> 216 -> 95 -> 81 -> 125 -> 432 -> 127 -> 415 -> 294 -> 131 -> 460 -> 35 -> 1

In [1]:
# Gerekli Kütüphanelerin İçe Aktarılması
import torch
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot
import random
import time
from typing import List, Tuple, Dict
from itertools import combinations
import math
import copy

# Bokeh'ı Jupyter Notebooks için Ayarlama
output_notebook()

# Google Drive'ı Bağlama (Google Colab için)
from google.colab import drive
drive.mount('/content/drive')

# Cihaz Kontrolü (GPU Kullanımı)
device_available = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Kullanılan Cihaz: {device_available}")


Mounted at /content/drive
Kullanılan Cihaz: cuda


In [4]:
import torch
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict, Optional
import time
from tqdm import tqdm
import random
import math
from concurrent.futures import ThreadPoolExecutor
import heapq
import plotly.graph_objects as go
import plotly.express as px

# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class TSPSolver:
    def __init__(self, file_path: str):
        self.cities_df = self._load_data(file_path)
        self.coords_tensor = self._prepare_coordinates()
        self.distance_matrix = self._compute_distance_matrix()
        self.num_cities = len(self.cities_df)

    def _load_data(self, file_path: str) -> pd.DataFrame:
        """Load and validate city coordinates."""
        try:
            data = pd.read_csv(file_path, header=None, delim_whitespace=True)
            data.columns = ['CityID', 'X_Coordinate', 'Y_Coordinate']
            data = data.astype({'CityID': int, 'X_Coordinate': float, 'Y_Coordinate': float})
            print(f"Loaded {len(data)} cities successfully")
            return data
        except Exception as e:
            raise RuntimeError(f"Error loading data: {e}")

    def _prepare_coordinates(self) -> torch.Tensor:
        """Convert coordinates to tensor format."""
        coordinates = self.cities_df[['X_Coordinate', 'Y_Coordinate']].values
        return torch.tensor(coordinates, dtype=torch.float32).to(device)

    def _compute_distance_matrix(self) -> torch.Tensor:
        """Compute Euclidean distance matrix efficiently."""
        with torch.no_grad():
            diff = self.coords_tensor.unsqueeze(1) - self.coords_tensor.unsqueeze(0)
            return torch.sqrt(torch.sum(diff ** 2, dim=2))

    def calculate_tour_length(self, tour: List[int]) -> float:
        """Calculate total tour length."""
        tour_tensor = torch.tensor(tour + [tour[0]], dtype=torch.long).to(device)
        return self.distance_matrix[tour_tensor[:-1], tour_tensor[1:]].sum().item()

    def plot_cities_interactive(self):
        """Plot the cities on an interactive Plotly scatter plot."""
        fig = go.Figure(data=go.Scatter(
            x=self.cities_df['X_Coordinate'],
            y=self.cities_df['Y_Coordinate'],
            mode='markers',
            marker=dict(color='blue', size=5, opacity=0.6),
            text=self.cities_df['CityID'],
            hoverinfo='text'
        ))

        fig.update_layout(
            title='City Locations',
            xaxis_title='X Coordinate',
            yaxis_title='Y Coordinate',
            template='plotly_white',
            width=1000,
            height=800
        )

        fig.show()

    def plot_tour_interactive(self, tour: List[int], title: str = "TSP Tour"):
        """Plot the tour on an interactive Plotly graph."""
        tour_extended = tour + [tour[0]]
        x = self.cities_df.loc[tour_extended, 'X_Coordinate'].values
        y = self.cities_df.loc[tour_extended, 'Y_Coordinate'].values

        fig = go.Figure()

        # Plot the tour lines
        fig.add_trace(go.Scatter(
            x=x,
            y=y,
            mode='lines',
            line=dict(color='orange', width=1),
            name='Tour Path'
        ))

        # Plot the cities
        fig.add_trace(go.Scatter(
            x=self.cities_df['X_Coordinate'],
            y=self.cities_df['Y_Coordinate'],
            mode='markers',
            marker=dict(color='blue', size=5, opacity=0.6),
            text=self.cities_df['CityID'],
            hoverinfo='text',
            name='Cities'
        ))

        # Highlight the start city
        fig.add_trace(go.Scatter(
            x=[x[0]],
            y=[y[0]],
            mode='markers',
            marker=dict(color='green', size=10, symbol='star'),
            name='Start City'
        ))

        # Highlight the end city
        fig.add_trace(go.Scatter(
            x=[x[-1]],
            y=[y[-1]],
            mode='markers',
            marker=dict(color='red', size=10, symbol='star'),
            name='End City'
        ))

        fig.update_layout(
            title=f"{title}\nTotal Distance: {self.calculate_tour_length(tour):.2f}",
            xaxis_title='X Coordinate',
            yaxis_title='Y Coordinate',
            template='plotly_white',
            width=1000,
            height=800
        )

        fig.show()

    def plot_comparison(self, results: Dict):
        """Compare algorithm performances using Plotly."""
        algorithms = list(results.keys())
        lengths = [result['length'] for result in results.values()]
        times = [result['time'] for result in results.values()]

        # Compare Tour Lengths
        fig1 = px.bar(x=algorithms, y=lengths,
                     labels={'x': 'Algorithm', 'y': 'Tour Length'},
                     title='Comparison of Tour Lengths',
                     text=[f"{l:.1f}" for l in lengths],
                     color=lengths,
                     color_continuous_scale='Viridis')

        fig1.update_traces(textposition='outside', marker_line_width=0)
        fig1.update_layout(uniformtext_minsize=8,
                          uniformtext_mode='hide',
                          width=1000,
                          height=600)
        fig1.show()

        # Compare Execution Times
        fig2 = px.bar(x=algorithms, y=times,
                     labels={'x': 'Algorithm', 'y': 'Time (s)'},
                     title='Comparison of Execution Times',
                     text=[f"{t:.2f}s" for t in times],
                     color=times,
                     color_continuous_scale='Plasma')

        fig2.update_traces(textposition='outside', marker_line_width=0)
        fig2.update_layout(uniformtext_minsize=8,
                          uniformtext_mode='hide',
                          width=1000,
                          height=600)
        fig2.show()

        # Detailed Comparison Table
        print("\nDetailed Comparison:")
        print("-" * 60)
        print(f"{'Algorithm':<30} {'Tour Length':<20} {'Time (s)':<15}")
        print("-" * 60)
        for algo, result in results.items():
            print(f"{algo:<30} {result['length']:<20.2f} {result['time']:<15.2f}")

    def k_nearest_neighbor_construction(self, k: int = 5, start_city: Optional[int] = None) -> List[int]:
        """Implementation of K-Nearest Neighbors algorithm with random start.

        Args:
            k: Number of nearest neighbors to consider at each step
            start_city: Starting city index (random if None)
        """
        if start_city is None:
            start_city = random.randint(0, self.num_cities - 1)

        unvisited = set(range(self.num_cities))
        current_city = start_city
        tour = [current_city]
        unvisited.remove(current_city)

        while unvisited:
            # Get distances to all unvisited cities
            distances = self.distance_matrix[current_city, list(unvisited)]
            unvisited_list = list(unvisited)

            # Find k nearest neighbors
            k = min(k, len(unvisited))  # Adjust k if fewer cities remain
            _, k_nearest_indices = torch.topk(distances, k, largest=False)
            k_nearest_cities = [unvisited_list[i] for i in k_nearest_indices]

            # Evaluate the impact of choosing each of the k nearest cities
            best_next_city = None
            min_impact = float('inf')

            for next_city in k_nearest_cities:
                # Calculate the immediate distance cost
                immediate_cost = self.distance_matrix[current_city, next_city]

                # Look ahead: calculate minimum distance to any remaining city
                remaining_cities = unvisited - {next_city}
                if remaining_cities:
                    future_costs = self.distance_matrix[next_city, list(remaining_cities)]
                    min_future_cost = future_costs.min().item()
                else:
                    # If this is the last city, consider distance back to start
                    min_future_cost = self.distance_matrix[next_city, tour[0]]

                # Total impact is immediate cost plus estimated future cost
                total_impact = immediate_cost + min_future_cost

                if total_impact < min_impact:
                    min_impact = total_impact
                    best_next_city = next_city

            # Add the best city to the tour
            tour.append(best_next_city)
            unvisited.remove(best_next_city)
            current_city = best_next_city

        return tour

    def three_opt_improvement(self, tour: List[int], max_iterations: int = 25) -> Tuple[List[int], float]:
        """Enhanced 3-opt local search with early stopping."""
        best_tour = tour.copy()
        best_length = self.calculate_tour_length(best_tour)
        improved = True
        iteration = 0

        while improved and iteration < max_iterations:
            improved = False
            for i in range(len(tour) - 2):
                for j in range(i + 2, len(tour) - 1):
                    for k in range(j + 2, len(tour)):
                        new_tour = self._three_opt_swap(best_tour.copy(), i, j, k)
                        new_length = self.calculate_tour_length(new_tour)

                        if new_length < best_length:
                            best_tour = new_tour
                            best_length = new_length
                            improved = True
                            break
                    if improved:
                        break
                if improved:
                    break
            iteration += 1

        return best_tour, best_length

    def _three_opt_swap(self, tour: List[int], i: int, j: int, k: int) -> List[int]:
        """Perform 3-opt swap operation."""
        segment1 = tour[i:j]
        segment2 = tour[j:k]
        segment3 = tour[k:] + tour[:i]

        possibilities = [
            segment1 + segment2 + segment3,
            segment1 + segment2[::-1] + segment3,
            segment1[::-1] + segment2 + segment3,
            segment1[::-1] + segment2[::-1] + segment3
        ]

        best_tour = min(possibilities, key=lambda t: self.calculate_tour_length(t))
        return best_tour

    def lin_kernighan(self, initial_tour: List[int], max_iterations: int = 50) -> Tuple[List[int], float]:
        """Implementation of Lin-Kernighan heuristic."""
        best_tour = initial_tour.copy()
        best_length = self.calculate_tour_length(best_tour)

        for _ in range(max_iterations):
            improved = False
            current_tour = best_tour.copy()

            for i in range(len(current_tour)):
                new_tour = self._find_improving_move(current_tour, i)
                if new_tour:
                    new_length = self.calculate_tour_length(new_tour)
                    if new_length < best_length:
                        best_tour = new_tour
                        best_length = new_length
                        improved = True
                        break

            if not improved:
                break

        return best_tour, best_length

    def _find_improving_move(self, tour: List[int], start_idx: int) -> Optional[List[int]]:
        """Find an improving k-opt move starting from given index."""
        current_city = tour[start_idx]
        visited = {current_city}
        path = [current_city]

        for _ in range(5):  # Limit the search depth
            best_gain = 0
            best_next = None

            for next_city in range(self.num_cities):
                if next_city not in visited:
                    gain = self._calculate_gain(path[-1], next_city, tour)
                    if gain > best_gain:
                        best_gain = gain
                        best_next = next_city

            if best_next is None:
                break

            path.append(best_next)
            visited.add(best_next)

            if len(path) >= 3:
                new_tour = self._create_new_tour(tour, path)
                if new_tour:
                    return new_tour

        return None

    def _calculate_gain(self, city1: int, city2: int, tour: List[int]) -> float:
        """Calculate the gain of connecting two cities."""
        idx1 = tour.index(city1)
        idx2 = tour.index(city2)

        old_length = (self.distance_matrix[city1, tour[(idx1 + 1) % self.num_cities]] +
                     self.distance_matrix[city2, tour[(idx2 + 1) % self.num_cities]])
        new_length = (self.distance_matrix[city1, city2] +
                     self.distance_matrix[tour[(idx1 + 1) % self.num_cities],
                                       tour[(idx2 + 1) % self.num_cities]])

        return old_length - new_length

    def _create_new_tour(self, old_tour: List[int], path: List[int]) -> Optional[List[int]]:
        """Create a new tour from the given path."""
        if len(path) < 3:
            return None

        new_tour = old_tour.copy()
        for i in range(len(path) - 1):
            idx1 = new_tour.index(path[i])
            idx2 = new_tour.index(path[i + 1])
            new_tour[idx1 + 1:idx2 + 1] = reversed(new_tour[idx1 + 1:idx2 + 1])

        return new_tour

    def solve(self) -> Tuple[List[int], float, Dict]:
        """Main solving method combining multiple approaches."""
        results = {}

        # 1. Generate initial solution using K-Nearest Neighbors
        print("Generating initial solution using K-Nearest Neighbors...")
        start_time = time.time()
        initial_tour = self.k_nearest_neighbor_construction(k=5)  # Using k=5 neighbors
        initial_length = self.calculate_tour_length(initial_tour)
        knn_time = time.time() - start_time
        results['K-Nearest Neighbors'] = {
            'tour': initial_tour,
            'length': initial_length,
            'time': knn_time
        }
        print(f"Initial tour length: {initial_length:.2f}")
        self.plot_tour_interactive(initial_tour, "Initial Tour - K-Nearest Neighbors")

        # 2. Apply 3-opt improvement
        print("\nApplying 3-opt improvement...")
        start_time = time.time()
        improved_tour, improved_length = self.three_opt_improvement(initial_tour)
        three_opt_time = time.time() - start_time
        results['3-Opt Improvement'] = {
            'tour': improved_tour,
            'length': improved_length,
            'time': three_opt_time
        }
        print(f"After 3-opt improvement: {improved_length:.2f}")
        self.plot_tour_interactive(improved_tour, "Improved Tour - 3-Opt")

        # 3. Apply Lin-Kernighan heuristic
        print("\nApplying Lin-Kernighan heuristic...")
        start_time = time.time()
        final_tour, final_length = self.lin_kernighan(improved_tour)
        lk_time = time.time() - start_time
        results['Lin-Kernighan'] = {
            'tour': final_tour,
            'length': final_length,
            'time': lk_time
        }
        print(f"Final tour length: {final_length:.2f}")
        self.plot_tour_interactive(final_tour, "Final Tour - Lin-Kernighan")

        # Compare all results
        print("\nComparing all algorithms...")
        self.plot_comparison(results)

        # Find the best result
        best_result = min(results.items(), key=lambda x: x[1]['length'])
        print(f"\nBest result achieved by {best_result[0]}:")
        print(f"Tour length: {best_result[1]['length']:.2f}")
        print(f"Computation time: {best_result[1]['time']:.2f} seconds")

        return best_result[1]['tour'], best_result[1]['length'], results

def main():
    """Main execution function."""
    # Specify the file path
    file_path = '/content/drive/My Drive/CityCoordinatesAlgortihm/coordinates.csv'

    print("\nInitializing TSP solver...")
    solver = TSPSolver(file_path)

    print("\nStarting optimization...")
    start_time = time.time()

    best_tour, best_length, results = solver.solve()

    total_time = time.time() - start_time
    print(f"\nTotal solution time: {total_time:.2f} seconds")

    # Save final results
    print("\nSaving results...")
    output_path = 'best_solution.txt'
    with open(output_path, 'w') as f:
        f.write(f"Best Tour Length: {best_length:.2f}\n")
        f.write(f"Solution Time: {total_time:.2f} seconds\n")
        f.write(f"Tour Sequence: {' -> '.join(map(str, best_tour))}\n")
    print(f"Results saved to '{output_path}'")

if __name__ == "__main__":
    main()

Using device: cuda

Initializing TSP solver...
Loaded 532 cities successfully

Starting optimization...
Generating initial solution using K-Nearest Neighbors...


<ipython-input-4-740af06859a2>:28: FutureWarning:

The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead



Initial tour length: 6875.24



Applying 3-opt improvement...
After 3-opt improvement: 6720.35



Applying Lin-Kernighan heuristic...
Final tour length: 5889.07



Comparing all algorithms...



Detailed Comparison:
------------------------------------------------------------
Algorithm                      Tour Length          Time (s)       
------------------------------------------------------------
K-Nearest Neighbors            6875.24              0.64           
3-Opt Improvement              6720.35              41.64          
Lin-Kernighan                  5889.07              146.18         

Best result achieved by Lin-Kernighan:
Tour length: 5889.07
Computation time: 146.18 seconds

Total solution time: 188.83 seconds

Saving results...
Results saved to 'best_solution.txt'
